In [2]:
class forecaster():
    def fit_predict_model(
        dataframe,
        interval_width = 0.8,
        changepoint_range = 0.8,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10,
        periods=60):

        m = Prophet(
            daily_seasonality = False, 
            yearly_seasonality = True, 
            weekly_seasonality = False,
            seasonality_mode = 'multiplicative',
            interval_width = interval_width,
            changepoint_range = changepoint_range,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale
        )

        m = m.fit(dataframe)
        future = m.make_future_dataframe(periods=periods, freq='MS', include_history=True)

        forecast = m.predict(future)
        forecast['fact'] = dataframe['y'].reset_index(drop = True)
        return forecast, m

    def detect_anomalies(forecast):
        forecasted = forecast[['ds','trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()
        forecasted['anomaly'] = 0
        forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
        forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1

        #anomaly importances
        forecasted['importance'] = 0
        forecasted.loc[forecasted['anomaly'] ==1, 'importance'] = (forecasted['fact'] - forecasted['yhat_upper'])/forecast['fact']
        forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] = (forecasted['yhat_lower'] - forecasted['fact'])/forecast['fact']

        return forecasted

    def evaluate_model(model):
        df_cv = cross_validation(
        model, 
        initial='731 days', 
        horizon='365 days',
        period='365 days',
        parallel = 'processes')
        df_p = performance_metrics(df_cv)
        mape = df_p['mape'].mean()
        rmse = df_p['rmse'].mean()
        print(f'MAPE: {mape}\nRMSE: {rmse}')
        
    def remove_anomalies(df, predictions):
        anomaly_list = predictions[predictions['anomaly'] != 0]['ds']
        return df[~df['ds'].isin(anomaly_list)]